# NDVI Example

We start as usual, by including the LCMAP Python Client, in order to retrieve Landsat data and invoke a sample model.

In [1]:
from lcmap.client import Client
import json
client = Client();

We specify a point and a time range that we would like to use in order to apply the NDVI function. The function is implemented in C, wrapped using Python ctypes, and packaged as a Linux container that is executed on LCMAP SEE infrastructure.

In [2]:
x = -2062080
y =  2952960
t1, t2 = "2002-05-01", "2003-01-01"

### Running a Model

Models may be explicitly invoked using the Python client. The model wrapper retrieves Landsat data from the LCMAP REST API using the specified parameters and saves results using the LCAMP REST API too.

In [3]:
run = client.models.ndvi.run(x, y, t1, t2)

Now we retrieve input and outputs of execution as tiles  like so:

In [4]:
_, sr3 = client.data.tiles("LANDSAT_7/ETM/sr_band3", x, y, t1, t2)
_, sr4 = client.data.tiles("LANDSAT_7/ETM/sr_band4", x, y, t1, t2)
_, ndvi = client.data.tiles("LCMAP/SEE/NDVI", x, y, t1, t2, unscale=True)

Notice that the acquisition date of each tile is the same...

In [5]:
sr3[1].acquired, sr4[1].acquired, ndvi[1].acquired

('2002-05-10T05:00:00Z', '2002-05-10T05:00:00Z', '2002-05-10T10:00:00Z')

### Comparing Results

Let's calculate NDVI again here and compare it against what was calculated by the C-library, they should be equal to the fourth decimal point (because of scaling).

In [6]:
a, b = sr3[1].data[0,0], sr4[1].data[0,0]
a, b
(b-a)/(b+a), ndvi[1].data[0,0]

(0.80740370185092558, 0.80740000000000001)

### Tile Source

Let's take a look at how the source of a Landsat ARD tile compares to the source of an NDVI tile produced by the LCMAP SEE. Notice that the source of an NDVI tile is a data structure that contains information about the inputs. This is only a starting point, but it contains a few attributes that are needed for provenance. Eventually, it will include information about the version of the model, and time of production.

In [7]:
sr3[0].source

'LE70470272002121PAC01'

In [8]:
sr4[0].source

'LE70470272002121PAC01'

In [9]:
some_tile = ndvi[1]
some_tile.source

'{"vis_acquired": "2002-05-10T05:00:00Z", "nir_y": 2952960, "vis_ubid": "LANDSAT_7/ETM/sr_band3", "nir_ubid": "LANDSAT_7/ETM/sr_band4", "job_id": "fake-job-id", "vis_y": 2952960, "vis_x": -2062080, "nir_x": -2062080, "nir_acquired": "2002-05-10T05:00:00Z"}'